# TPS-Aug-2022

In [1]:
class Config:
    NB = '107'
    dataset_NB = '106'

    raw_data_dir = '../data/raw/'
    processed_data_dir = '../data/processed/'
    interim_dir = '../data/interim/'
    submission_dir = '../data/submission/'

    random_seed = 42
    n_folds = 5

    row_id = 'id'
    target = 'failure'

## Import libralies

In [2]:
import gc
import warnings
warnings.filterwarnings('ignore')

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plotly_template = dict(
    layout=go.Layout(
        template='plotly_dark',
        font=dict(
            family="Franklin Gothic",
            size=12
        ),
        height=500,
        width=1000,
    )
)


color_palette = {
    'Bin': ['#016CC9','#E876A3'],
    'Cat5': ['#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E'],
}

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import make_pipeline

## Load data

In [5]:
df_train = pd.read_pickle(f"../data/processed/nb{Config.dataset_NB}_train.pkl", compression='zip')
df_test = pd.read_pickle(f"../data/processed/nb{Config.dataset_NB}_test.pkl", compression='zip')

df_dataset = pd.concat(objs=[df_train, df_test], axis=0).reset_index(drop=True)
df_dataset

,id,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,attribute_0_material_5,attribute_0_material_7,attribute_1_material_5,attribute_1_material_6,attribute_1_material_7,attribute_1_material_8,mes_0x1,mes_0-1,mes_0x2,mes_0-2,mes_0x3,mes_0-3,mes_0x4,mes_0-4,mes_0x5,mes_0-5,mes_0x6,mes_0-6,mes_0x7,mes_0-7,mes_0x8,mes_0-8,mes_0x9,mes_0-9,mes_0x10,mes_0-10,mes_0x11,mes_0-11,mes_0x12,mes_0-12,mes_0x13,mes_0-13,mes_0x14,mes_0-14,mes_0x15,mes_0-15,mes_0x16,mes_0-16,mes_0x17,mes_0-17,mes_1x2,mes_1-2,mes_1x3,mes_1-3,mes_1x4,mes_1-4,mes_1x5,mes_1-5,mes_1x6,mes_1-6,mes_1x7,mes_1-7,mes_1x8,mes_1-8,mes_1x9,mes_1-9,mes_1x10,mes_1-10,mes_1x11,mes_1-11,mes_1x12,mes_1-12,mes_1x13,mes_1-13,mes_1x14,mes_1-14,mes_1x15,mes_1-15,mes_1x16,mes_1-16,mes_1x17,mes_1-17,mes_2x3,mes_2-3,mes_2x4,mes_2-4,mes_2x5,mes_2-5,mes_2x6,mes_2-6,mes_2x7,mes_2-7,mes_2x8,mes_2-8,mes_2x9,mes_2-9,mes_2x10,mes_2-10,mes_2x11,mes_2-11,mes_2x12,mes_2-12,mes_2x13,mes_2-13,mes_2x14,mes_2-14,mes_2x15,mes_2-15,mes_2x16,mes_2-16,mes_2x17,mes_2-17,mes_3x4,mes_3-4,mes_3x5,mes_3-5,mes_3x6,mes_3-6,mes_3x7,mes_3-7,mes_3x8,mes_3-8,mes_3x9,mes_3-9,mes_3x10,mes_3-10,mes_3x11,mes_3-11,mes_3x12,mes_3-12,mes_3x13,mes_3-13,mes_3x14,mes_3-14,mes_3x15,mes_3-15,mes_3x16,mes_3-16,mes_3x17,mes_3-17,mes_4x5,mes_4-5,mes_4x6,mes_4-6,mes_4x7,mes_4-7,mes_4x8,mes_4-8,mes_4x9,mes_4-9,mes_4x10,mes_4-10,mes_4x11,mes_4-11,mes_4x12,mes_4-12,mes_4x13,mes_4-13,mes_4x14,mes_4-14,mes_4x15,mes_4-15,mes_4x16,mes_4-16,mes_4x17,mes_4-17,mes_5x6,mes_5-6,mes_5x7,mes_5-7,mes_5x8,mes_5-8,mes_5x9,mes_5-9,mes_5x10,mes_5-10,mes_5x11,mes_5-11,mes_5x12,mes_5-12,mes_5x13,mes_5-13,mes_5x14,mes_5-14,mes_5x15,mes_5-15,mes_5x16,mes_5-16,mes_5x17,mes_5-17,mes_6x7,mes_6-7,mes_6x8,mes_6-8,mes_6x9,mes_6-9,mes_6x10,mes_6-10,mes_6x11,mes_6-11,mes_6x12,mes_6-12,mes_6x13,mes_6-13,mes_6x14,mes_6-14,mes_6x15,mes_6-15,mes_6x16,mes_6-16,mes_6x17,mes_6-17,mes_7x8,mes_7-8,mes_7x9,mes_7-9,mes_7x10,mes_7-10,mes_7x11,mes_7-11,mes_7x12,mes_7-12,mes_7x13,mes_7-13,mes_7x14,mes_7-14,mes_7x15,mes_7-15,mes_7x16,mes_7-16,mes_7x17,mes_7-17,mes_8x9,mes_8-9,mes_8x10,mes_8-10,mes_8x11,mes_8-11,mes_8x12,mes_8-12,mes_8x13,mes_8-13,mes_8x14,mes_8-14,mes_8x15,mes_8-15,mes_8x16,mes_8-16,mes_8x17,mes_8-17,mes_9x10,mes_9-10,mes_9x11,mes_9-11,mes_9x12,mes_9-12,mes_9x13,mes_9-13,mes_9x14,mes_9-14,mes_9x15,mes_9-15,mes_9x16,mes_9-16,mes_9x17,mes_9-17,mes_10x11,mes_10-11,mes_10x12,mes_10-12,mes_10x13,mes_10-13,mes_10x14,mes_10-14,mes_10x15,mes_10-15,mes_10x16,mes_10-16,mes_10x17,mes_10-17,mes_11x12,mes_11-12,mes_11x13,mes_11-13,mes_11x14,mes_11-14,mes_11x15,mes_11-15,mes_11x16,mes_11-16,mes_11x17,mes_11-17,mes_12x13,mes_12-13,mes_12x14,mes_12-14,mes_12x15,mes_12-15,mes_12x16,mes_12-16,mes_12x17,mes_12-17,mes_13x14,mes_13-14,mes_13x15,mes_13-15,mes_13x16,mes_13-16,mes_13x17,mes_13-17,mes_14x15,mes_14-15,mes_14x16,mes_14-16,mes_14x17,mes_14-17,mes_15x16,mes_15-16,mes_15x17,mes_15-17,mes_16x17,mes_16-17
0,0,0.249005,1.0,0.2,0.368421,0.40,0.285714,0.545796,0.647744,0.238275,0.842618,0.504651,0.717593,0.352201,0.463325,0.321003,0.984393,0.388937,0.491051,0.231624,0.280078,0.606211,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.345679,0.068966,0.227642,0.206897,0.386289,0.524124,0.408039,0.372247,0.350102,0.422997,0.419101,0.606285,0.382058,0.321670,0.403836,0.572638,0.356491,0.255693,0.374058,0.432615,0.354424,0.446684,0.489764,0.545037,0.368338,0.400479,0.380874,0.442356,0.333582,0.308807,0.340318,0.361573,0.409804,0.606493,0.215488,0.333333,0.407426,0.481071,0.429327,0.336612,0.371381,0.390235,0.444019,0.542671,0.404876,0.277941,0.426771,0.551061,0.377386,0.204258,0.394135,0.401656,0.375029,0.416869,0.518386,0.523165,0.385644,0.372205,0.401642,0.409818,0.348535,0.273513,0.358666,0.328670,0.434108,0.607195,0.269421,0.628597,0.279960,0.563834,0

## 正規化&標準化

In [6]:
def clip_outlier(df, col):
    '''clip outlier
    '''

    Q1 = np.percentile(df[col], 25)
    Q3 = np.percentile(df[col], 75)

    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    df.loc[:, col] = np.maximum(df[col], Q1 - outlier_step)
    df.loc[:, col] = np.minimum(df[col], Q3 + outlier_step)

    return df


def clip_manual_outlier(df, col, percentile, upper=True):
    '''clip manual outlier
    '''

    Q = np.quantile(df[col], percentile)

    if upper:
        df.loc[:, col] = np.minimum(df[col], Q)
    else:
        df.loc[:, col] = np.maximum(df[col], Q)

    return df


def preprocess_dataset(df):
    """Preprocess the data (select columns and scale)
    ### MinMaxScaler: 正規化（値を0~1の範囲に）
    ### RobustScalar: 正規化（ただし、四分位範囲を分母とする）
    ### StandardScaler: 標準化（平均との差をとり標準偏差で割る）
    """
    # preproc = make_pipeline(MinMaxScaler(), StandardScaler(with_std=False))
    preproc = make_pipeline(RobustScaler(), StandardScaler(with_std=False))
    df_f = pd.DataFrame(preproc.fit_transform(df), columns=df.columns, index=df.index)

    return df_f

def normalize_dataset(df):
    """Preprocess the data (select columns and scale)
    ### MinMaxScaler: 正規化（値を0~1の範囲に）
    """
    preproc = make_pipeline(MinMaxScaler())
    df_f = pd.DataFrame(preproc.fit_transform(df), columns=df.columns, index=df.index)

    return df_f

def standardize_dataset(df):
    """Preprocess the data (select columns and scale)
    ### StandardScaler: 標準化（平均との差をとり標準偏差で割る）
    """
    preproc = make_pipeline(StandardScaler(with_std=False))
    df_f = pd.DataFrame(preproc.fit_transform(df), columns=df.columns, index=df.index)

    return df_f

In [7]:
feature_list = [col for col in df_dataset.columns if col not in [Config.row_id, Config.target]]

In [8]:
df_dataset[feature_list] = standardize_dataset(df_dataset[feature_list])
df_dataset.describe()

,id,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,attribute_0_material_5,attribute_0_material_7,attribute_1_material_5,attribute_1_material_6,attribute_1_material_7,attribute_1_material_8,mes_0x1,mes_0-1,mes_0x2,mes_0-2,mes_0x3,mes_0-3,mes_0x4,mes_0-4,mes_0x5,mes_0-5,mes_0x6,mes_0-6,mes_0x7,mes_0-7,mes_0x8,mes_0-8,mes_0x9,mes_0-9,mes_0x10,mes_0-10,mes_0x11,mes_0-11,mes_0x12,mes_0-12,mes_0x13,mes_0-13,mes_0x14,mes_0-14,mes_0x15,mes_0-15,mes_0x16,mes_0-16,mes_0x17,mes_0-17,mes_1x2,mes_1-2,mes_1x3,mes_1-3,mes_1x4,mes_1-4,mes_1x5,mes_1-5,mes_1x6,mes_1-6,mes_1x7,mes_1-7,mes_1x8,mes_1-8,mes_1x9,mes_1-9,mes_1x10,mes_1-10,mes_1x11,mes_1-11,mes_1x12,mes_1-12,mes_1x13,mes_1-13,mes_1x14,mes_1-14,mes_1x15,mes_1-15,mes_1x16,mes_1-16,mes_1x17,mes_1-17,mes_2x3,mes_2-3,mes_2x4,mes_2-4,mes_2x5,mes_2-5,mes_2x6,mes_2-6,mes_2x7,mes_2-7,mes_2x8,mes_2-8,mes_2x9,mes_2-9,mes_2x10,mes_2-10,mes_2x11,mes_2-11,mes_2x12,mes_2-12,mes_2x13,mes_2-13,mes_2x14,mes_2-14,mes_2x15,mes_2-15,mes_2x16,mes_2-16,mes_2x17,mes_2-17,mes_3x4,mes_3-4,mes_3x5,mes_3-5,mes_3x6,mes_3-6,mes_3x7,mes_3-7,mes_3x8,mes_3-8,mes_3x9,mes_3-9,mes_3x10,mes_3-10,mes_3x11,mes_3-11,mes_3x12,mes_3-12,mes_3x13,mes_3-13,mes_3x14,mes_3-14,mes_3x15,mes_3-15,mes_3x16,mes_3-16,mes_3x17,mes_3-17,mes_4x5,mes_4-5,mes_4x6,mes_4-6,mes_4x7,mes_4-7,mes_4x8,mes_4-8,mes_4x9,mes_4-9,mes_4x10,mes_4-10,mes_4x11,mes_4-11,mes_4x12,mes_4-12,mes_4x13,mes_4-13,mes_4x14,mes_4-14,mes_4x15,mes_4-15,mes_4x16,mes_4-16,mes_4x17,mes_4-17,mes_5x6,mes_5-6,mes_5x7,mes_5-7,mes_5x8,mes_5-8,mes_5x9,mes_5-9,mes_5x10,mes_5-10,mes_5x11,mes_5-11,mes_5x12,mes_5-12,mes_5x13,mes_5-13,mes_5x14,mes_5-14,mes_5x15,mes_5-15,mes_5x16,mes_5-16,mes_5x17,mes_5-17,mes_6x7,mes_6-7,mes_6x8,mes_6-8,mes_6x9,mes_6-9,mes_6x10,mes_6-10,mes_6x11,mes_6-11,mes_6x12,mes_6-12,mes_6x13,mes_6-13,mes_6x14,mes_6-14,mes_6x15,mes_6-15,mes_6x16,mes_6-16,mes_6x17,mes_6-17,mes_7x8,mes_7-8,mes_7x9,mes_7-9,mes_7x10,mes_7-10,mes_7x11,mes_7-11,mes_7x12,mes_7-12,mes_7x13,mes_7-13,mes_7x14,mes_7-14,mes_7x15,mes_7-15,mes_7x16,mes_7-16,mes_7x17,mes_7-17,mes_8x9,mes_8-9,mes_8x10,mes_8-10,mes_8x11,mes_8-11,mes_8x12,mes_8-12,mes_8x13,mes_8-13,mes_8x14,mes_8-14,mes_8x15,mes_8-15,mes_8x16,mes_8-16,mes_8x17,mes_8-17,mes_9x10,mes_9-10,mes_9x11,mes_9-11,mes_9x12,mes_9-12,mes_9x13,mes_9-13,mes_9x14,mes_9-14,mes_9x15,mes_9-15,mes_9x16,mes_9-16,mes_9x17,mes_9-17,mes_10x11,mes_10-11,mes_10x12,mes_10-12,mes_10x13,mes_10-13,mes_10x14,mes_10-14,mes_10x15,mes_10-15,mes_10x16,mes_10-16,mes_10x17,mes_10-17,mes_11x12,mes_11-12,mes_11x13,mes_11-13,mes_11x14,mes_11-14,mes_11x15,mes_11-15,mes_11x16,mes_11-16,mes_11x17,mes_11-17,mes_12x13,mes_12-13,mes_12x14,mes_12-14,mes_12x15,mes_12-15,mes_12x16,mes_12-16,mes_12x17,mes_12-17,mes_13x14,mes_13-14,mes_13x15,mes_13-15,mes_13x16,mes_13-16,mes_13x17,mes_13-17,mes_14x15,mes_14-15,mes_14x16,mes_14-16,mes_14x17,mes_14-17,mes_15x16,mes_15-16,mes_15x17,mes_15-17,mes_16x17,mes_16-17
count,47345.000000,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,26570.000000,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,47345.0,47345.0,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+04,4.734500e+

In [9]:
df_dataset

,id,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,attribute_0_material_5,attribute_0_material_7,attribute_1_material_5,attribute_1_material_6,attribute_1_material_7,attribute_1_material_8,mes_0x1,mes_0-1,mes_0x2,mes_0-2,mes_0x3,mes_0-3,mes_0x4,mes_0-4,mes_0x5,mes_0-5,mes_0x6,mes_0-6,mes_0x7,mes_0-7,mes_0x8,mes_0-8,mes_0x9,mes_0-9,mes_0x10,mes_0-10,mes_0x11,mes_0-11,mes_0x12,mes_0-12,mes_0x13,mes_0-13,mes_0x14,mes_0-14,mes_0x15,mes_0-15,mes_0x16,mes_0-16,mes_0x17,mes_0-17,mes_1x2,mes_1-2,mes_1x3,mes_1-3,mes_1x4,mes_1-4,mes_1x5,mes_1-5,mes_1x6,mes_1-6,mes_1x7,mes_1-7,mes_1x8,mes_1-8,mes_1x9,mes_1-9,mes_1x10,mes_1-10,mes_1x11,mes_1-11,mes_1x12,mes_1-12,mes_1x13,mes_1-13,mes_1x14,mes_1-14,mes_1x15,mes_1-15,mes_1x16,mes_1-16,mes_1x17,mes_1-17,mes_2x3,mes_2-3,mes_2x4,mes_2-4,mes_2x5,mes_2-5,mes_2x6,mes_2-6,mes_2x7,mes_2-7,mes_2x8,mes_2-8,mes_2x9,mes_2-9,mes_2x10,mes_2-10,mes_2x11,mes_2-11,mes_2x12,mes_2-12,mes_2x13,mes_2-13,mes_2x14,mes_2-14,mes_2x15,mes_2-15,mes_2x16,mes_2-16,mes_2x17,mes_2-17,mes_3x4,mes_3-4,mes_3x5,mes_3-5,mes_3x6,mes_3-6,mes_3x7,mes_3-7,mes_3x8,mes_3-8,mes_3x9,mes_3-9,mes_3x10,mes_3-10,mes_3x11,mes_3-11,mes_3x12,mes_3-12,mes_3x13,mes_3-13,mes_3x14,mes_3-14,mes_3x15,mes_3-15,mes_3x16,mes_3-16,mes_3x17,mes_3-17,mes_4x5,mes_4-5,mes_4x6,mes_4-6,mes_4x7,mes_4-7,mes_4x8,mes_4-8,mes_4x9,mes_4-9,mes_4x10,mes_4-10,mes_4x11,mes_4-11,mes_4x12,mes_4-12,mes_4x13,mes_4-13,mes_4x14,mes_4-14,mes_4x15,mes_4-15,mes_4x16,mes_4-16,mes_4x17,mes_4-17,mes_5x6,mes_5-6,mes_5x7,mes_5-7,mes_5x8,mes_5-8,mes_5x9,mes_5-9,mes_5x10,mes_5-10,mes_5x11,mes_5-11,mes_5x12,mes_5-12,mes_5x13,mes_5-13,mes_5x14,mes_5-14,mes_5x15,mes_5-15,mes_5x16,mes_5-16,mes_5x17,mes_5-17,mes_6x7,mes_6-7,mes_6x8,mes_6-8,mes_6x9,mes_6-9,mes_6x10,mes_6-10,mes_6x11,mes_6-11,mes_6x12,mes_6-12,mes_6x13,mes_6-13,mes_6x14,mes_6-14,mes_6x15,mes_6-15,mes_6x16,mes_6-16,mes_6x17,mes_6-17,mes_7x8,mes_7-8,mes_7x9,mes_7-9,mes_7x10,mes_7-10,mes_7x11,mes_7-11,mes_7x12,mes_7-12,mes_7x13,mes_7-13,mes_7x14,mes_7-14,mes_7x15,mes_7-15,mes_7x16,mes_7-16,mes_7x17,mes_7-17,mes_8x9,mes_8-9,mes_8x10,mes_8-10,mes_8x11,mes_8-11,mes_8x12,mes_8-12,mes_8x13,mes_8-13,mes_8x14,mes_8-14,mes_8x15,mes_8-15,mes_8x16,mes_8-16,mes_8x17,mes_8-17,mes_9x10,mes_9-10,mes_9x11,mes_9-11,mes_9x12,mes_9-12,mes_9x13,mes_9-13,mes_9x14,mes_9-14,mes_9x15,mes_9-15,mes_9x16,mes_9-16,mes_9x17,mes_9-17,mes_10x11,mes_10-11,mes_10x12,mes_10-12,mes_10x13,mes_10-13,mes_10x14,mes_10-14,mes_10x15,mes_10-15,mes_10x16,mes_10-16,mes_10x17,mes_10-17,mes_11x12,mes_11-12,mes_11x13,mes_11-13,mes_11x14,mes_11-14,mes_11x15,mes_11-15,mes_11x16,mes_11-16,mes_11x17,mes_11-17,mes_12x13,mes_12-13,mes_12x14,mes_12-14,mes_12x15,mes_12-15,mes_12x16,mes_12-16,mes_12x17,mes_12-17,mes_13x14,mes_13-14,mes_13x15,mes_13-15,mes_13x16,mes_13-16,mes_13x17,mes_13-17,mes_14x15,mes_14-15,mes_14x16,mes_14-16,mes_14x17,mes_14-17,mes_15x16,mes_15-16,mes_15x17,mes_15-17,mes_16x17,mes_16-17
0,0,-0.248981,0.453992,-0.356502,-0.021676,-0.026445,-0.152416,0.046469,0.148113,-0.261730,0.342805,0.004773,0.217511,-0.147992,-0.036281,-0.178646,0.486223,-0.113305,-0.008071,-0.270465,-0.219982,0.106514,0.0,-0.333277,0.333277,-0.329285,-0.335241,0.0,0.0,-0.019432,-0.276588,-0.124094,-0.104314,-0.016490,0.028459,0.004069,0.022071,-0.052671,-0.051183,0.016797,0.104576,-0.020869,-0.029927,0.000838,0.068643,-0.046939,-0.091189,-0.028107,-0.000183,-0.050036,-0.044803,0.088048,0.195593,-0.036946,-0.022105,-0.022719,0.009684,-0.071967,-0.096418,-0.064011,-0.074601,0.013393,0.106988,-0.137046,-0.033344,-0.020665,0.031664,0.000804,0.000280,-0.059039,-0.050868,0.014508,0.104817,-0.025231,-0.057329,-0.002560,0.072553,-0.052897,-0.127749,-0.032992,0.000383,-0.055968,-0.042937,0.090734,0.187331,-0.043551,-0.023671,-0.027736,0.010649,-0.

## Save Dataset

In [10]:
## Separate train dataset and test dataset
train_len = len(df_train)

train = df_dataset[:train_len]
test = df_dataset[train_len:]
test.drop(columns=[Config.target], inplace=True)

In [11]:
pd.to_pickle(train, Config.processed_data_dir + f"nb{Config.NB}_train.pkl", compression='zip')
pd.to_pickle(test, Config.processed_data_dir + f"nb{Config.NB}_test.pkl", compression='zip')

## 検証メモ